In [1]:
import pandas as pd

# 파일 경로 지정
file_path = "/Users/skku_aws30/Desktop/ne0033sy/MIMIC-IV-Preprocessing/vitals/rr_filtered_with_vent_merged.csv"

# CSV 파일 로드 및 필요한 컬럼만 선택
df = pd.read_csv(file_path, parse_dates=['charttime'])
df_filtered = df[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'valuenum']]

# subject_id 기준으로 charttime 정렬
df_sorted = df_filtered.sort_values(by=['subject_id', 'charttime'])

# 같은 파일명으로 저장
df_sorted.to_csv(file_path, index=False)

# 결측치 확인 (전체 컬럼 대상)
missing_summary = df_sorted.isnull().sum()
print("컬럼별 결측치 수:\n", missing_summary)


컬럼별 결측치 수:
 subject_id    0
hadm_id       0
stay_id       0
charttime     0
valuenum      0
dtype: int64


## RR 균등시계열(bin) 생성

In [5]:
# 생존 환자는 outtime, 사망환자는 deathtime을 기준으로 bin 생성

import pandas as pd
import numpy as np
from datetime import timedelta
# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
rr_data = pd.read_csv('vitals/rr_filtered_with_vent_merged.csv')
# 2. datetime 변환
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
rr_data['charttime'] = pd.to_datetime(rr_data['charttime']).dt.tz_localize(None)
# 3. 코호트와 RR 데이터 병합
rr_merged = rr_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')
# 4. ICU 재원 기간 내 데이터만 필터링
mask = (rr_merged['charttime'] >= rr_merged['intime']) & (rr_merged['charttime'] <= rr_merged['outtime'])
rr_filtered = rr_merged[mask].copy()
# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in rr_filtered['stay_id'].unique():
    patient_data = rr_filtered[rr_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]
    
    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 RR 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]['valuenum']
        
        # 대표값 계산
        if len(hour_data) > 0:
            rr_mean = hour_data.mean()
            rr_last = hour_data.iloc[-1]
            rr_min = hour_data.min()
            rr_max = hour_data.max()
        else:
            rr_mean = np.nan
            rr_last = np.nan
            rr_min = np.nan
            rr_max = np.nan
        
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'rr_mean': rr_mean,
            'rr_last': rr_last,
            'rr_min': rr_min,
            'rr_max': rr_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('rr_hourly_bins.csv', index=False)

## GCS 균등시계열(bin) 생성

In [6]:
import pandas as pd
import numpy as np
from datetime import timedelta
# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
gcs_data = pd.read_csv('vitals/gcs_filtered_with_sadatedflag.csv')
# 2. datetime 변환
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
gcs_data['charttime'] = pd.to_datetime(gcs_data['charttime']).dt.tz_localize(None)
# 3. 코호트와 GCS 데이터 병합
gcs_merged = gcs_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')
# 4. ICU 재원 기간 내 데이터만 필터링
mask = (gcs_merged['charttime'] >= gcs_merged['intime']) & (gcs_merged['charttime'] <= gcs_merged['outtime'])
gcs_filtered = gcs_merged[mask].copy()
# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in gcs_filtered['stay_id'].unique():
    patient_data = gcs_filtered[gcs_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]
    
    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 GCS 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]
        
        # 대표값 계산
        if len(hour_data) > 0:
            gcs_mean = hour_data['gcs_total'].mean()
            gcs_last = hour_data['gcs_total'].iloc[-1]
            gcs_min = hour_data['gcs_total'].min()
            gcs_max = hour_data['gcs_total'].max()
            # sedated_flag가 한번이라도 1이면 1, 아니면 0
            sedated_flag = 1 if (hour_data['sedated_flag'] == 1).any() else 0
        else:
            gcs_mean = np.nan
            gcs_last = np.nan
            gcs_min = np.nan
            gcs_max = np.nan
            sedated_flag = np.nan
        
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'gcs_mean': gcs_mean,
            'gcs_last': gcs_last,
            'gcs_min': gcs_min,
            'gcs_max': gcs_max,
            'sedated_flag': sedated_flag
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/gcs_hourly_bins.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
bp_data = pd.read_csv('vitals/----.csv')
# 2. datetime 변환
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
bp_data['charttime'] = pd.to_datetime(bp_data['charttime']).dt.tz_localize(None)
# 3. 코호트와 BP 데이터 병합
bp_merged = bp_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')
# 4. ICU 재원 기간 내 데이터만 필터링
mask = (bp_merged['charttime'] >= bp_merged['intime']) & (bp_merged['charttime'] <= bp_merged['outtime'])
bp_filtered = bp_merged[mask].copy()
# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in bp_filtered['stay_id'].unique():
    patient_data = bp_filtered[bp_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]
    
    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 SBP, DBP, MAP 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]
        
        # 대표값 계산
        if len(hour_data) > 0:
            sbp_mean = hour_data['final_sbp'].mean()
            sbp_last = hour_data['final_sbp'].iloc[-1]
            sbp_min = hour_data['final_sbp'].min()
            sbp_max = hour_data['final_sbp'].max()
            dbp_mean = hour_data['final_dbp'].mean()
            dbp_last = hour_data['final_dbp'].iloc[-1]
            dbp_min = hour_data['final_dbp'].min()
            dbp_max = hour_data['final_dbp'].max()
            map_mean = hour_data['final_map'].mean()
            map_last = hour_data['final_map'].iloc[-1]
            map_min = hour_data['final_map'].min()
            map_max = hour_data['final_map'].max()
        else:
            sbp_mean = np.nan
            sbp_last = np.nan
            sbp_min = np.nan
            sbp_max = np.nan
            dbp_mean = np.nan
            dbp_last = np.nan
            dbp_min = np.nan
            dbp_max = np.nan
            map_mean = np.nan
            map_last = np.nan
            map_min = np.nan
            map_max = np.nan
            
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'sbp_mean': sbp_mean,
            'sbp_last': sbp_last,
            'sbp_min': sbp_min,
            'sbp_max': sbp_max,
            'dbp_mean': dbp_mean,
            'dbp_last': dbp_last,
            'dbp_min': dbp_min,
            'dbp_max': dbp_max,
            'map_mean': map_mean,
            'map_last': map_last,
            'map_min': map_min,
            'map_max': map_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/bp_hourly_bins.csv', index=False)

            

## HR 균등시계열 생성

In [7]:
import pandas as pd
import numpy as np
from datetime import timedelta
# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
hr_data = pd.read_csv('vitals/hr_filtered_merged.csv')
# 2. 데이터 전처리
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
hr_data['charttime'] = pd.to_datetime(hr_data['charttime']).dt.tz_localize(None)
# 3. 코호트와 HR 데이터 병합
hr_merged = hr_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')
# 4. ICU 재원 기간 내 데이터만 필터링
mask = (hr_merged['charttime'] >= hr_merged['intime']) & (hr_merged['charttime'] <= hr_merged['outtime'])
hr_filtered = hr_merged[mask].copy()
# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in hr_filtered['stay_id'].unique():
    patient_data = hr_filtered[hr_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]

    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 HR 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]['valuenum']
        
        # 대표값 계산
        if len(hour_data) > 0:
            hr_mean = hour_data.mean()
            hr_last = hour_data.iloc[-1]
            hr_min = hour_data.min()
            hr_max = hour_data.max()
        else:
            hr_mean = np.nan
            hr_last = np.nan
            hr_min = np.nan
            hr_max = np.nan
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'hr_mean': hr_mean,
            'hr_last': hr_last,
            'hr_min': hr_min,
            'hr_max': hr_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/hr_hourly_bins.csv', index=False)


## Temp 균등 시계열 생성

In [8]:
import pandas as pd
import numpy as np
from datetime import timedelta
# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
temp_data = pd.read_csv('vitals/temp_filtered_merged.csv')
# 2. 데이터 전처리
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
temp_data['charttime'] = pd.to_datetime(temp_data['charttime']).dt.tz_localize(None)
# 3. 코호트와 TEMP 데이터 병합
temp_merged = temp_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')
# 4. ICU 재원 기간 내 데이터만 필터링
mask = (temp_merged['charttime'] >= temp_merged['intime']) & (temp_merged['charttime'] <= temp_merged['outtime'])
temp_filtered = temp_merged[mask].copy()
# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in temp_filtered['stay_id'].unique():
    patient_data = temp_filtered[temp_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]

    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 TEMP 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]['temperature_celsius']
        
        # 대표값 계산 (소수점 셋째자리까지)
        if len(hour_data) > 0:
            temp_mean = round(hour_data.mean(), 3)
            temp_last = round(hour_data.iloc[-1], 3)
            temp_min = round(hour_data.min(), 3)
            temp_max = round(hour_data.max(), 3)
        else:
            temp_mean = np.nan
            temp_last = np.nan
            temp_min = np.nan
            temp_max = np.nan
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'temp_mean': temp_mean,
            'temp_last': temp_last,
            'temp_min': temp_min,
            'temp_max': temp_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/temp_hourly_bins.csv', index=False)

## SpO2 균등 시계열 생성

In [9]:
import pandas as pd
import numpy as np
from datetime import timedelta

# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
spo2_data = pd.read_csv('vitals/spo2_filtered_merged.csv')

# 2. 데이터 전처리
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
spo2_data['charttime'] = pd.to_datetime(spo2_data['charttime']).dt.tz_localize(None)

# 3. 코호트와 SpO2 데이터 병합
spo2_merged = spo2_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')

# 4. ICU 재원 기간 내 데이터만 필터링
mask = (spo2_merged['charttime'] >= spo2_merged['intime']) & (spo2_merged['charttime'] <= spo2_merged['outtime'])
spo2_filtered = spo2_merged[mask].copy()

# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in spo2_filtered['stay_id'].unique():
    patient_data = spo2_filtered[spo2_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]

    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 SpO2 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]['valuenum']
        
        # 대표값 계산 (소수점 셋째자리까지)
        if len(hour_data) > 0:
            spo2_mean = hour_data.mean()
            spo2_last = hour_data.iloc[-1]
            spo2_min = hour_data.min()
            spo2_max = hour_data.max()
        else:
            spo2_mean = np.nan
            spo2_last = np.nan
            spo2_min = np.nan
            spo2_max = np.nan
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'spo2_mean': spo2_mean,
            'spo2_last': spo2_last,
            'spo2_min': spo2_min,
            'spo2_max': spo2_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/spo2_hourly_bins.csv', index=False)

## BP (SBP/DBP/MAP) 균등 시계열 생성

In [10]:
import pandas as pd
import numpy as np
from datetime import timedelta

# 1. 데이터 로드
cohort = pd.read_csv('cohort.csv')
bp_data = pd.read_csv('vitals/bp_final_merged.csv')

# 2. 데이터 전처리
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])
bp_data['charttime'] = pd.to_datetime(bp_data['charttime']).dt.tz_localize(None)

# 3. 코호트와 BP 데이터 병합
bp_merged = bp_data.merge(cohort[['stay_id', 'intime', 'outtime', 'deathtime']], on='stay_id', how='inner')

# 4. ICU 재원 기간 내 데이터만 필터링
mask = (bp_merged['charttime'] >= bp_merged['intime']) & (bp_merged['charttime'] <= bp_merged['outtime'])
bp_filtered = bp_merged[mask].copy()

# 5. 1시간 단위 bin 생성 및 대표값 계산
result_list = []
for stay_id in bp_filtered['stay_id'].unique():
    patient_data = bp_filtered[bp_filtered['stay_id'] == stay_id].copy()
    patient_cohort = cohort[cohort['stay_id'] == stay_id].iloc[0]

    intime = patient_cohort['intime']
    outtime = patient_cohort['outtime']
    deathtime = patient_cohort['deathtime']
    icu_los_hours = patient_cohort['icu_los_hours']
    
    # bin 생성 종료 시간 결정
    if pd.notna(deathtime):
        end_time = deathtime  # 사망 환자는 deathtime까지
    else:
        end_time = outtime    # 생존 환자는 outtime까지
    
    # intime을 기준으로 1시간 단위 bin 생성
    total_hours = int(np.ceil(icu_los_hours))
    
    for hour in range(total_hours):
        # 시간 범위 정의
        bin_start = intime + timedelta(hours=hour)
        bin_end = intime + timedelta(hours=hour+1)
        
        # end_time을 넘지 않도록 조정
        if bin_end > end_time:
            bin_end = end_time
        
        # bin_start가 end_time을 넘으면 중단
        if bin_start >= end_time:
            break
        
        # 해당 시간 범위의 BP 데이터 추출
        hour_mask = (patient_data['charttime'] >= bin_start) & (patient_data['charttime'] < bin_end)
        hour_data = patient_data[hour_mask]
        
        # 대표값 계산 (반올림 없이 원본 값 사용)
        if len(hour_data) > 0:
            sbp_mean = hour_data['final_sbp'].mean()
            sbp_last = hour_data['final_sbp'].iloc[-1]
            sbp_min = hour_data['final_sbp'].min()
            sbp_max = hour_data['final_sbp'].max()
            
            dbp_mean = hour_data['final_dbp'].mean()
            dbp_last = hour_data['final_dbp'].iloc[-1]
            dbp_min = hour_data['final_dbp'].min()
            dbp_max = hour_data['final_dbp'].max()
            
            map_mean = hour_data['final_map'].mean()
            map_last = hour_data['final_map'].iloc[-1]
            map_min = hour_data['final_map'].min()
            map_max = hour_data['final_map'].max()
        else:
            sbp_mean = sbp_last = sbp_min = sbp_max = np.nan
            dbp_mean = dbp_last = dbp_min = dbp_max = np.nan
            map_mean = map_last = map_min = map_max = np.nan
        # 결과 저장
        result_list.append({
            'subject_id': patient_cohort['subject_id'],
            'hadm_id': patient_cohort['hadm_id'],
            'stay_id': stay_id,
            'hour_from_intime': hour,
            'bin_start': bin_start,
            'bin_end': bin_end,
            'sbp_mean': sbp_mean,
            'sbp_last': sbp_last,
            'sbp_min': sbp_min,
            'sbp_max': sbp_max,
            'dbp_mean': dbp_mean,
            'dbp_last': dbp_last,
            'dbp_min': dbp_min,
            'dbp_max': dbp_max,
            'map_mean': map_mean,
            'map_last': map_last,
            'map_min': map_min,
            'map_max': map_max
        })
# 6. 결과를 DataFrame으로 변환 및 저장
result_df = pd.DataFrame(result_list)
result_df.to_csv('hourly_bins/bp_hourly_bins.csv', index=False)

## Vitals 병합

In [1]:
import pandas as pd
from functools import reduce

# 파일 경로와 피쳐 접두어 매핑
file_info = [
    ('hourly_bins/bp_hourly_bins.csv', ['sbp', 'dbp', 'map']),
    ('hourly_bins/hr_hourly_bins.csv', ['hr']),
    ('hourly_bins/temp_hourly_bins.csv', ['temp']),
    ('hourly_bins/spo2_hourly_bins.csv', ['spo2']),
    ('hourly_bins/rr_hourly_bins.csv', ['rr']),
    ('hourly_bins/gcs_hourly_bins.csv', ['gcs']) # sedated_flag 포함
]

# 공통 key
merge_keys = ['subject_id', 'hadm_id', 'stay_id', 'hour_from_intime', 'bin_start', 'bin_end']

# 파일별 데이터프레임 로드
dfs = []
for path, _ in file_info:
    df = pd.read_csv(path)
    dfs.append(df)

# 순차적으로 inner 병합
merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dfs)

# 저장
merged.to_csv('hourly_bins/all_features_hourly_bins_inner.csv', index=False)
print("병합 완료! 결과 파일: hourly_bins/all_features_hourly_bins_inner.csv")

병합 완료! 결과 파일: hourly_bins/all_features_hourly_bins_inner.csv


# sliding window 구성
- 관찰 윈도우 18h, 예측 윈도우 6h, 예측 간격(시작시점) 8h

In [2]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
print("1. 데이터 로딩...")
data = pd.read_csv("hourly_bins/all_features_hourly_bins_inner.csv")
cohort = pd.read_csv("cohort.csv")

# datetime 변환
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])

print(f"   - 전체 bin 수: {len(data)}")
print(f"   - 환자 수: {data['subject_id'].nunique()}")

# 2. 윈도우 설정
obs_window_hours = 18  # 관찰 윈도우
pred_window_hours = 6   # 예측 윈도우
pred_interval_hours = 8 # 예측 간격

# 3. 코호트 정보와 병합
data_with_cohort = data.merge(cohort[['subject_id', 'stay_id', 'intime', 'outtime', 'deathtime']], 
                              on=['subject_id', 'stay_id'], how='inner')

# 4. subject_id 기준으로 그룹화 및 정렬
print("2. 데이터 그룹화 및 정렬...")
data_grouped = data_with_cohort.groupby('subject_id')

# 피처 컬럼 정의
feature_cols = [col for col in data.columns 
               if col not in ['subject_id', 'stay_id', 'hour_from_intime', 'subject_id', 'hadm_id', 'bin_start', 'bin_end']]

# 5. 벡터화된 슬라이딩 윈도우 생성
print("3. 슬라이딩 윈도우 생성...")

windows = []
total_subjects = len(data_grouped)
processed_subjects = 0

for subject_id, subject_data in data_grouped:
    processed_subjects += 1
    if processed_subjects % 1000 == 0:
        print(f"   진행률: {processed_subjects}/{total_subjects} ({processed_subjects/total_subjects*100:.1f}%)")
    
    # 환자별 데이터 정렬
    subject_data = subject_data.sort_values('hour_from_intime').reset_index(drop=True)
    
    # 환자별 코호트 정보
    stay_id = subject_data['stay_id'].iloc[0]
    intime = subject_data['intime'].iloc[0]
    outtime = subject_data['outtime'].iloc[0]  
    deathtime = subject_data['deathtime'].iloc[0]
    
    # 최대 시간
    max_hour = subject_data['hour_from_intime'].max()
    
    # 가능한 모든 윈도우 시작점 계산 (벡터화)
    possible_starts = np.arange(0, max_hour + 1 - obs_window_hours - pred_window_hours + 1, pred_interval_hours)
    
    # 각 시작점에 대해 윈도우 생성
    for window_id, start_hour in enumerate(possible_starts):
        obs_start = start_hour
        obs_end = start_hour + obs_window_hours
        pred_start = obs_end
        pred_end = obs_end + pred_window_hours
        
        # 관찰 윈도우 데이터 추출 (벡터화)
        obs_mask = (subject_data['hour_from_intime'] >= obs_start) & (subject_data['hour_from_intime'] < obs_end)
        obs_window = subject_data[obs_mask]
        
        # 관찰 윈도우 완성도 검사
        expected_obs_hours = obs_window_hours
        actual_obs_hours = len(obs_window)
        obs_completeness = actual_obs_hours / expected_obs_hours
        
        if obs_completeness < 0.3:
            continue
        
        # 사망 라벨 결정 (벡터화)
        pred_window_start_time = intime + pd.Timedelta(hours=pred_start)
        pred_window_end_time = intime + pd.Timedelta(hours=pred_end)
        
        death_in_pred_window = 0
        if pd.notna(deathtime):
            if pred_window_start_time <= deathtime <= pred_window_end_time:
                death_in_pred_window = 1
        
        # 시퀀스 데이터 생성 (벡터화)
        # 18시간 범위의 hour_from_intime 생성
        hour_range = np.arange(obs_start, obs_end)
        
        # 실제 데이터와 매핑을 위한 인덱스 생성
        obs_window_indexed = obs_window.set_index('hour_from_intime')
        
        # 18시간 시퀀스 데이터 생성
        sequence_data = []
        for hour in hour_range:
            if hour in obs_window_indexed.index:
                # 해당 시간의 데이터가 있음
                hour_features = obs_window_indexed.loc[hour, feature_cols].values
            else:
                # 해당 시간의 데이터가 없음 - NaN으로 채움
                hour_features = np.full(len(feature_cols), np.nan)
            
            sequence_data.append(hour_features.tolist())
        
        # 윈도우 정보 저장
        window_info = {
            'subject_id': subject_id,
            'stay_id': stay_id,
            'window_id': window_id,
            'obs_start_hour': obs_start,
            'obs_end_hour': obs_end,
            'pred_start_hour': pred_start,
            'pred_end_hour': pred_end,
            'obs_completeness': obs_completeness,
            'death_in_pred_window': death_in_pred_window,
            'sequence': sequence_data,
            'feature_names': feature_cols
        }
        
        windows.append(window_info)

print(f"\n4. 슬라이딩 윈도우 생성 완료!")

# 5. 결과를 DataFrame으로 변환
print("5. 결과 정리...")
windows_df = pd.DataFrame(windows)

# 통계 출력
print(f"   - 생성된 총 윈도우 수: {len(windows_df)}")
print(f"   - 윈도우가 있는 환자 수: {windows_df['subject_id'].nunique()}")
print(f"   - 환자별 평균 윈도우 수: {len(windows_df) / windows_df['subject_id'].nunique():.1f}")
print(f"   - 사망 라벨 1인 윈도우: {windows_df['death_in_pred_window'].sum()} ({windows_df['death_in_pred_window'].mean()*100:.1f}%)")
print(f"   - 평균 관찰 윈도우 완성도: {windows_df['obs_completeness'].mean():.3f}")

# 6. 결과 저장
print("6. 결과 저장...")
windows_df.to_csv('sliding_windows_18h_6h.csv', index=False)
print("슬라이딩 윈도우 생성 완료!")

1. 데이터 로딩...
   - 전체 bin 수: 4414240
   - 환자 수: 47339
2. 데이터 그룹화 및 정렬...
3. 슬라이딩 윈도우 생성...
   진행률: 1000/47339 (2.1%)
   진행률: 2000/47339 (4.2%)
   진행률: 3000/47339 (6.3%)
   진행률: 4000/47339 (8.4%)
   진행률: 5000/47339 (10.6%)
   진행률: 6000/47339 (12.7%)
   진행률: 7000/47339 (14.8%)
   진행률: 8000/47339 (16.9%)
   진행률: 9000/47339 (19.0%)
   진행률: 10000/47339 (21.1%)
   진행률: 11000/47339 (23.2%)
   진행률: 12000/47339 (25.3%)
   진행률: 13000/47339 (27.5%)
   진행률: 14000/47339 (29.6%)
   진행률: 15000/47339 (31.7%)
   진행률: 16000/47339 (33.8%)
   진행률: 17000/47339 (35.9%)
   진행률: 18000/47339 (38.0%)
   진행률: 19000/47339 (40.1%)
   진행률: 20000/47339 (42.2%)
   진행률: 21000/47339 (44.4%)
   진행률: 22000/47339 (46.5%)
   진행률: 23000/47339 (48.6%)
   진행률: 24000/47339 (50.7%)
   진행률: 25000/47339 (52.8%)
   진행률: 26000/47339 (54.9%)
   진행률: 27000/47339 (57.0%)
   진행률: 28000/47339 (59.1%)
   진행률: 29000/47339 (61.3%)
   진행률: 30000/47339 (63.4%)
   진행률: 31000/47339 (65.5%)
   진행률: 32000/47339 (67.6%)
   진행률: 33000/47339 (69.7%)

In [5]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
print("1. 데이터 로딩...")
data = pd.read_csv("hourly_bins/all_features_hourly_bins_inner.csv")
cohort = pd.read_csv("cohort.csv")

# datetime 변환
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])

print(f"   - 전체 bin 수: {len(data)}")
print(f"   - 환자 수: {data['subject_id'].nunique()}")

# 2. 윈도우 설정
obs_window_hours = 18  # 관찰 윈도우
pred_window_hours = 6   # 예측 윈도우
pred_interval_hours = 8 # 예측 간격

# 3. 코호트 정보와 병합
data_with_cohort = data.merge(cohort[['subject_id', 'stay_id', 'intime', 'outtime', 'deathtime']], 
                              on=['subject_id', 'stay_id'], how='inner')

# 4. subject_id 기준으로 그룹화 및 정렬
print("2. 데이터 그룹화 및 정렬...")
data_grouped = data_with_cohort.groupby('subject_id')

# 피처 컬럼 정의
feature_cols = [col for col in data.columns 
               if col not in ['subject_id', 'stay_id', 'hour_from_intime', 'subject_id', 'hadm_id', 'bin_start', 'bin_end']]

# 5. 벡터화된 슬라이딩 윈도우 생성
print("3. 슬라이딩 윈도우 생성...")

windows = []
sequences = []  # ✅ sequences 리스트 초기화
total_subjects = len(data_grouped)
processed_subjects = 0

for subject_id, subject_data in data_grouped:
    processed_subjects += 1
    if processed_subjects % 1000 == 0:
        print(f"   진행률: {processed_subjects}/{total_subjects} ({processed_subjects/total_subjects*100:.1f}%)")
    
    # 환자별 데이터 정렬
    subject_data = subject_data.sort_values('hour_from_intime').reset_index(drop=True)
    
    # 환자별 코호트 정보
    stay_id = subject_data['stay_id'].iloc[0]
    intime = subject_data['intime'].iloc[0]
    outtime = subject_data['outtime'].iloc[0]  
    deathtime = subject_data['deathtime'].iloc[0]
    
    # 최대 시간
    max_hour = subject_data['hour_from_intime'].max()
    
    # 가능한 모든 윈도우 시작점 계산 (벡터화)
    possible_starts = np.arange(0, max_hour + 1 - obs_window_hours - pred_window_hours + 1, pred_interval_hours)
    
    # 각 시작점에 대해 윈도우 생성
    for window_id, start_hour in enumerate(possible_starts):
        obs_start = start_hour
        obs_end = start_hour + obs_window_hours
        pred_start = obs_end
        pred_end = obs_end + pred_window_hours
        
        # 관찰 윈도우 데이터 추출 (벡터화)
        obs_mask = (subject_data['hour_from_intime'] >= obs_start) & (subject_data['hour_from_intime'] < obs_end)
        obs_window = subject_data[obs_mask]
        
        # ===== 수정된 결측률 계산 부분 =====
        # 시퀀스 데이터 생성을 위한 준비
        hour_range = np.arange(obs_start, obs_end)
        obs_window_indexed = obs_window.set_index('hour_from_intime')
        
        # 18시간 시퀀스 데이터 생성 및 결측률 계산
        sequence_data = []
        total_values = 0
        missing_values = 0
        
        for hour in hour_range:
            if hour in obs_window_indexed.index:
                # 해당 시간의 데이터가 있음
                hour_features = obs_window_indexed.loc[hour, feature_cols].values
                # NaN 개수 계산
                missing_in_hour = pd.isna(hour_features).sum()
                missing_values += missing_in_hour
                total_values += len(hour_features)
            else:
                # 해당 시간의 데이터가 없음 - 전체가 결측
                hour_features = np.full(len(feature_cols), np.nan)
                missing_values += len(feature_cols)
                total_values += len(feature_cols)
            
            sequence_data.append(hour_features.tolist())
        
        # 실제 결측률 계산
        obs_completeness = 1 - (missing_values / total_values) if total_values > 0 else 0
        
        # 결측률 기준으로 필터링 (70% 이상 결측이면 제외)
        if obs_completeness < 0.3:
            continue
        
        # 사망 라벨 결정 (벡터화)
        pred_window_start_time = intime + pd.Timedelta(hours=pred_start)
        pred_window_end_time = intime + pd.Timedelta(hours=pred_end)
        
        death_in_pred_window = 0
        if pd.notna(deathtime):
            if pred_window_start_time <= deathtime <= pred_window_end_time:
                death_in_pred_window = 1
        
        # ✅ 윈도우 메타데이터 저장 (sequence 제거)
        window_info = {
            'subject_id': subject_id,
            'stay_id': stay_id,
            'window_id': window_id,
            'obs_start_hour': obs_start,
            'obs_end_hour': obs_end,
            'pred_start_hour': pred_start,
            'pred_end_hour': pred_end,
            'obs_completeness': obs_completeness,
            'total_values': total_values,
            'missing_values': missing_values,
            'death_in_pred_window': death_in_pred_window
            # ✅ 'sequence' 제거 - 메타데이터만 저장
        }
        
        windows.append(window_info)
        
        # ✅ 시퀀스 데이터를 별도 리스트에 저장 (윈도우별로)
        sequences.append(np.array(sequence_data))  # shape: (18, n_features)

print(f"\n4. 슬라이딩 윈도우 생성 완료!")

# 5. 결과를 DataFrame으로 변환
print("5. 결과 정리...")
windows_df = pd.DataFrame(windows)

# 통계 출력
print(f"   - 생성된 총 윈도우 수: {len(windows_df)}")
print(f"   - 윈도우가 있는 환자 수: {windows_df['subject_id'].nunique()}")
print(f"   - 환자별 평균 윈도우 수: {len(windows_df) / windows_df['subject_id'].nunique():.1f}")
print(f"   - 사망 라벨 1인 윈도우: {windows_df['death_in_pred_window'].sum()} ({windows_df['death_in_pred_window'].mean()*100:.1f}%)")
print(f"   - 평균 관찰 윈도우 완성도: {windows_df['obs_completeness'].mean():.3f}")
print(f"   - 완성도 분포:")
print(f"     * 0.9 이상: {(windows_df['obs_completeness'] >= 0.9).sum()} ({(windows_df['obs_completeness'] >= 0.9).mean()*100:.1f}%)")
print(f"     * 0.7-0.9: {((windows_df['obs_completeness'] >= 0.7) & (windows_df['obs_completeness'] < 0.9)).sum()}")
print(f"     * 0.5-0.7: {((windows_df['obs_completeness'] >= 0.5) & (windows_df['obs_completeness'] < 0.7)).sum()}")
print(f"     * 0.3-0.5: {((windows_df['obs_completeness'] >= 0.3) & (windows_df['obs_completeness'] < 0.5)).sum()}")

# 6. 결과 저장
print("6. 결과 저장...")

# ✅ 메타데이터만 CSV로 저장 (sequence 컬럼 제거됨)
windows_df.to_csv('sliding_windows_metadata.csv', index=False)

# ✅ 시퀀스 데이터를 3D numpy array로 저장
sequences_array = np.array(sequences)  # shape: (n_windows, 18, n_features)
np.save('sliding_windows_sequences.npy', sequences_array)

# ✅ 피처 이름 저장
with open('feature_names.txt', 'w') as f:
    f.write('\n'.join(feature_cols))

print("슬라이딩 윈도우 생성 완료!")
print(f"   - 시퀀스 데이터 shape: {sequences_array.shape}")

# 7. 추가 분석을 위한 샘플 출력
if len(windows_df) > 0:
    print("\n7. 샘플 윈도우 정보:")
    sample_window = windows_df.iloc[0]
    print(f"   - Subject ID: {sample_window['subject_id']}")
    print(f"   - 완성도: {sample_window['obs_completeness']:.3f}")
    print(f"   - 총 값 수: {sample_window['total_values']}")
    print(f"   - 결측 값 수: {sample_window['missing_values']}")
    print(f"   - 첫 번째 윈도우 시퀀스 shape: {sequences_array[0].shape}")
    print(f"   - 피처 수: {len(feature_cols)}")

# 8. 데이터 검증
print("\n8. 데이터 검증:")
print(f"   - 윈도우 개수 일치: {len(windows_df)} == {len(sequences_array)}")
print(f"   - 시퀀스 길이 확인: {sequences_array.shape[1]} == {obs_window_hours}")
print(f"   - 피처 수 확인: {sequences_array.shape[2]} == {len(feature_cols)}")

1. 데이터 로딩...
   - 전체 bin 수: 4414240
   - 환자 수: 47339
2. 데이터 그룹화 및 정렬...
3. 슬라이딩 윈도우 생성...
   진행률: 1000/47339 (2.1%)
   진행률: 2000/47339 (4.2%)
   진행률: 3000/47339 (6.3%)
   진행률: 4000/47339 (8.4%)
   진행률: 5000/47339 (10.6%)
   진행률: 6000/47339 (12.7%)
   진행률: 7000/47339 (14.8%)
   진행률: 8000/47339 (16.9%)
   진행률: 9000/47339 (19.0%)
   진행률: 10000/47339 (21.1%)
   진행률: 11000/47339 (23.2%)
   진행률: 12000/47339 (25.3%)
   진행률: 13000/47339 (27.5%)
   진행률: 14000/47339 (29.6%)
   진행률: 15000/47339 (31.7%)
   진행률: 16000/47339 (33.8%)
   진행률: 17000/47339 (35.9%)
   진행률: 18000/47339 (38.0%)
   진행률: 19000/47339 (40.1%)
   진행률: 20000/47339 (42.2%)
   진행률: 21000/47339 (44.4%)
   진행률: 22000/47339 (46.5%)
   진행률: 23000/47339 (48.6%)
   진행률: 24000/47339 (50.7%)
   진행률: 25000/47339 (52.8%)
   진행률: 26000/47339 (54.9%)
   진행률: 27000/47339 (57.0%)
   진행률: 28000/47339 (59.1%)
   진행률: 29000/47339 (61.3%)
   진행률: 30000/47339 (63.4%)
   진행률: 31000/47339 (65.5%)
   진행률: 32000/47339 (67.6%)
   진행률: 33000/47339 (69.7%)

In [8]:
death_windows = windows_df[windows_df['death_in_pred_window']==1]
removed_due_to_completeness = 3500 - death_windows['subject_id'].nunique()
print(f"removed_due_to_completeness: {removed_due_to_completeness}")


removed_due_to_completeness: 3073


In [ ]:
import pandas as pd
import numpy as np

# ===== 파라미터 =====
obs_window_hours = 18
pred_window_hours = 6
pred_interval_hours = 8
extra_hours_after_pred = 2  # 예측 구간 종료 직후 추가 허용 시간

# ===== 데이터 로드 =====
print("1. 데이터 로딩...")
data = pd.read_csv("hourly_bins/all_features_hourly_bins_inner.csv")
cohort = pd.read_csv("cohort.csv")

cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])

# 병합
data_with_cohort = data.merge(
    cohort[['subject_id','stay_id','intime','outtime','deathtime']],
    on=['subject_id','stay_id'], how='inner'
)

# 피처 컬럼 (메타데이터 제외)
exclude_cols = ['subject_id', 'stay_id', 'hour_from_intime', 'hadm_id', 'bin_start', 'bin_end']
feature_cols = [c for c in data.columns if c not in exclude_cols]

# 그룹화
data_grouped = data_with_cohort.groupby('subject_id')

# ===== 윈도우 루프 =====
windows = []
sequences = []
old_label_count = 0
new_label_count = 0

for subject_id, subject_data in data_grouped:
    subject_data = subject_data.sort_values('hour_from_intime').reset_index(drop=True)
    
    stay_id = subject_data['stay_id'].iloc[0]
    intime = subject_data['intime'].iloc[0]
    deathtime = subject_data['deathtime'].iloc[0]
    max_hour = subject_data['hour_from_intime'].max()

    possible_starts = np.arange(0, max_hour + 1 - obs_window_hours - pred_window_hours + 1,
                                pred_interval_hours)

    for window_id, obs_start in enumerate(possible_starts):
        obs_end = obs_start + obs_window_hours
        pred_start = obs_end
        pred_end = obs_end + pred_window_hours
        
        # 시간 계산
        obs_start_time  = intime + pd.Timedelta(hours=obs_start)
        obs_end_time    = intime + pd.Timedelta(hours=obs_end)
        pred_start_time = intime + pd.Timedelta(hours=pred_start)
        pred_end_time   = intime + pd.Timedelta(hours=pred_end)
        
        # ========== 기존 라벨 (old_label) ==========
        old_label = 0
        if pd.notna(deathtime):
            if pred_start_time <= deathtime <= pred_end_time:
                old_label = 1
        
        # ========== 새로운 라벨링 로직 ==========
        remove_window = False
        new_label = 0
        if pd.notna(deathtime):
            # 관찰 중 사망 → 제거
            if obs_start_time <= deathtime <= obs_end_time:
                remove_window = True
            # 예측 구간 내 사망
            elif pred_start_time <= deathtime <= pred_end_time:
                new_label = 1
            # 예측 구간 종료 직후 2시간 내 사망
            elif pred_end_time < deathtime <= pred_end_time + pd.Timedelta(hours=extra_hours_after_pred):
                new_label = 1
        
        if remove_window:
            continue
        
        # ===== 관찰 데이터 & completeness 계산 =====
        obs_mask = (subject_data['hour_from_intime'] >= obs_start) & \
                   (subject_data['hour_from_intime'] < obs_end)
        obs_window = subject_data[obs_mask]
        
        hour_range = np.arange(obs_start, obs_end)
        obs_window_indexed = obs_window.set_index('hour_from_intime')
        
        sequence_data = []
        total_values = 0
        missing_values = 0
        
        for hour in hour_range:
            if hour in obs_window_indexed.index:
                hour_features = obs_window_indexed.loc[hour, feature_cols].values
                missing_in_hour = pd.isna(hour_features).sum()
            else:
                hour_features = np.full(len(feature_cols), np.nan)
                missing_in_hour = len(feature_cols)
            sequence_data.append(hour_features.tolist())
            missing_values += missing_in_hour
            total_values += len(feature_cols)
        
        obs_completeness = 1 - (missing_values / total_values) if total_values > 0 else 0
        
        # 라벨 0인 경우에만 completeness 기준 적용
        if (new_label == 0) and (obs_completeness < 0.7):
            continue
        
        # 저장
        windows.append({
            'subject_id': subject_id,
            'stay_id': stay_id,
            'window_id': window_id,
            'obs_start_hour': obs_start,
            'obs_end_hour': obs_end,
            'pred_start_hour': pred_start,
            'pred_end_hour': pred_end,
            'obs_completeness': obs_completeness,
            'death_in_pred_window_old': old_label,
            'death_in_pred_window_new': new_label
        })
        sequences.append(np.array(sequence_data))
        
        old_label_count += old_label
        new_label_count += new_label

# ===== 결과 저장 및 리포트 =====
windows_df = pd.DataFrame(windows)
sequences_array = np.array(sequences)

windows_df.to_csv('sliding_windows_metadata.csv', index=False)
np.save('sliding_windows_sequences.npy', sequences_array)

print("\n===== 라벨 변경 리포트 =====")
print(f"기존 라벨 1 윈도우 수: {old_label_count}")
print(f"새 로직 라벨 1 윈도우 수: {new_label_count}")
print(f"증가 수: {new_label_count - old_label_count} ({(new_label_count - old_label_count) / old_label_count * 100 if old_label_count else 0:.1f}%)")
print(f"전체 윈도우 수: {len(windows_df)}")
print(f"라벨 1 비율(기존): {old_label_count / len(windows_df) * 100:.2f}%")
print(f"라벨 1 비율(새로): {new_label_count / len(windows_df) * 100:.2f}%")


1. 데이터 로딩...

===== 라벨 변경 리포트 =====
기존 라벨 1 윈도우 수: 433
새 로직 라벨 1 윈도우 수: 1276
증가 수: 843 (194.7%)
전체 윈도우 수: 352165
라벨 1 비율(기존): 0.12%
라벨 1 비율(새로): 0.36%


# 결측값 확인 및 컬럼 정리

In [12]:
import pandas as pd
import numpy as np

print("===== 생성된 파일 분석 =====")

# ===== 1. 생성된 파일들 로드 및 분석 =====
# 메타데이터 파일 로드
metadata = pd.read_csv('sliding_windows_metadata.csv')
print(f"메타데이터 파일 크기: {metadata.shape}")
print(f"컬럼: {list(metadata.columns)}")
print("\n메타데이터 샘플:")
print(metadata.head())

# 시퀀스 데이터 로드
sequences = np.load('sliding_windows_sequences.npy')
print(f"\n시퀀스 데이터 형태: {sequences.shape}")
print(f"시퀀스 데이터 타입: {sequences.dtype}")
print(f"각 윈도우 크기: {sequences.shape[1]}시간 × {sequences.shape[2]}피처")

# 메타데이터 통계
print(f"\n===== 메타데이터 통계 =====")
print(f"총 윈도우 수: {len(metadata)}")
print(f"환자 수: {metadata['subject_id'].nunique()}")
print(f"평균 관찰 완전성: {metadata['obs_completeness'].mean():.3f}")
print(f"기존 라벨 1 비율: {metadata['death_in_pred_window_old'].mean():.3f}")
print(f"새 라벨 1 비율: {metadata['death_in_pred_window_new'].mean():.3f}")

print("\n관찰 완전성 분포:")
print(metadata['obs_completeness'].describe())

# ===== 2. 원본 데이터 로드 (결측값 처리를 위해) =====
print("\n===== 결측값 처리 시작 =====")
original_data = pd.read_csv("hourly_bins/all_features_hourly_bins_inner.csv")
print(f"원본 데이터 크기: {original_data.shape}")

# 메타데이터 컬럼 제외하고 피처 컬럼만 추출
exclude_cols = ['subject_id', 'stay_id', 'hour_from_intime', 'hadm_id', 'bin_start', 'bin_end']
feature_cols = [c for c in original_data.columns if c not in exclude_cols]
print(f"처리할 피처 수: {len(feature_cols)}")

# 결측값 현황 확인
missing_before = original_data[feature_cols].isnull().sum().sum()
total_values = len(original_data) * len(feature_cols)
print(f"처리 전 결측값: {missing_before:,} / {total_values:,} ({missing_before/total_values*100:.2f}%)")

# ===== 3. 환자별 결측값 보간 =====
print("\n환자별 결측값 보간 중...")
processed_data = original_data.copy()

# 환자별로 그룹화하여 처리
for subject_id in processed_data['subject_id'].unique():
    patient_mask = processed_data['subject_id'] == subject_id
    patient_data = processed_data[patient_mask].copy()
    
    # 시간 순서로 정렬
    patient_data = patient_data.sort_values('hour_from_intime')
    
    # 각 피처별로 ffill 적용
    for col in feature_cols:
        # ffill (forward fill) 적용
        patient_data[col] = patient_data[col].ffill()
        
        # ffill로도 채워지지 않은 값들 (첫 번째 값이 NaN인 경우)은 해당 환자의 평균으로 채움
        if patient_data[col].isnull().any():
            patient_mean = patient_data[col].mean()
            if not np.isnan(patient_mean):  # 해당 환자에게 해당 피처의 값이 하나라도 있는 경우
                patient_data[col] = patient_data[col].fillna(patient_mean)
            else:  # 해당 환자에게 해당 피처의 값이 전혀 없는 경우, 전체 평균 사용
                global_mean = processed_data[col].mean()
                patient_data[col] = patient_data[col].fillna(global_mean)
    
    # 원본 데이터에 반영
    processed_data.loc[patient_mask, feature_cols] = patient_data[feature_cols].values

# 결측값 처리 결과 확인
missing_after = processed_data[feature_cols].isnull().sum().sum()
print(f"처리 후 결측값: {missing_after:,} / {total_values:,} ({missing_after/total_values*100:.2f}%)")
print(f"제거된 결측값: {missing_before - missing_after:,}")

# ===== 4. 불필요한 컬럼 제거 =====
print("\n===== 컬럼 정리 =====")

# 제거할 컬럼들 정의
cols_to_remove = ['temp_min', 'temp_max', 'gcs_min', 'gcs_max']

# old_label 관련 컬럼들도 찾아서 제거 (컬럼명에 'old' 또는 'old_label'이 포함된 것들)
old_label_cols = [col for col in processed_data.columns if 'old' in col.lower()]
cols_to_remove.extend(old_label_cols)

# 실제 존재하는 컬럼만 필터링
existing_cols_to_remove = [col for col in cols_to_remove if col in processed_data.columns]
print(f"제거할 컬럼들: {existing_cols_to_remove}")

# 컬럼 제거
if existing_cols_to_remove:
    processed_data = processed_data.drop(columns=existing_cols_to_remove)
    print(f"제거된 컬럼 수: {len(existing_cols_to_remove)}")
else:
    print("제거할 컬럼이 존재하지 않습니다.")

print(f"최종 데이터 크기: {processed_data.shape}")

# ===== 5. 메타데이터에서도 old_label 관련 컬럼 제거 =====
print("\n메타데이터에서 old_label 관련 컬럼 제거...")
metadata_old_cols = [col for col in metadata.columns if 'old' in col.lower()]
if metadata_old_cols:
    metadata_cleaned = metadata.drop(columns=metadata_old_cols)
    print(f"메타데이터에서 제거된 컬럼: {metadata_old_cols}")
else:
    metadata_cleaned = metadata.copy()
    print("메타데이터에서 제거할 old_label 관련 컬럼이 없습니다.")

# ===== 6. 처리된 데이터로 새로운 시퀀스 생성 =====
print("\n===== 새로운 시퀀스 데이터 생성 =====")

# 새로운 피처 컬럼 목록 (제거된 컬럼 반영)
new_exclude_cols = ['subject_id', 'stay_id', 'hour_from_intime', 'hadm_id', 'bin_start', 'bin_end']
new_feature_cols = [c for c in processed_data.columns if c not in new_exclude_cols]
print(f"새로운 피처 수: {len(new_feature_cols)}")

# 코호트 데이터 다시 로드
cohort = pd.read_csv("cohort.csv")
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])

# 병합
processed_data_with_cohort = processed_data.merge(
    cohort[['subject_id','stay_id','intime','outtime','deathtime']],
    on=['subject_id','stay_id'], how='inner'
)

# 새로운 시퀀스 생성
obs_window_hours = 18
new_sequences = []

processed_grouped = processed_data_with_cohort.groupby('subject_id')

for _, row in metadata_cleaned.iterrows():
    subject_id = row['subject_id']
    obs_start = row['obs_start_hour']
    obs_end = row['obs_end_hour']
    
    # 해당 환자 데이터 가져오기
    subject_data = processed_grouped.get_group(subject_id).sort_values('hour_from_intime').reset_index(drop=True)
    
    # 관찰 구간 데이터 추출
    obs_mask = (subject_data['hour_from_intime'] >= obs_start) & \
               (subject_data['hour_from_intime'] < obs_end)
    obs_window = subject_data[obs_mask]
    
    # 시간별 데이터 생성
    hour_range = np.arange(obs_start, obs_end)
    obs_window_indexed = obs_window.set_index('hour_from_intime')
    
    sequence_data = []
    for hour in hour_range:
        if hour in obs_window_indexed.index:
            hour_features = obs_window_indexed.loc[hour, new_feature_cols].values
        else:
            # 해당 시간의 데이터가 없는 경우 NaN으로 채움 (이미 보간했으므로 거의 발생하지 않음)
            hour_features = np.full(len(new_feature_cols), np.nan)
        sequence_data.append(hour_features.tolist())
    
    new_sequences.append(np.array(sequence_data))

new_sequences_array = np.array(new_sequences)
print(f"새로운 시퀀스 데이터 형태: {new_sequences_array.shape}")

# ===== 7. 최종 파일 저장 =====
print("\n===== 최종 파일 저장 =====")

# 처리된 원본 데이터 저장
processed_data.to_csv('processed_hourly_data.csv', index=False)
print("처리된 시간별 데이터 저장: processed_hourly_data.csv")

# 정리된 메타데이터 저장
metadata_cleaned.to_csv('cleaned_sliding_windows_metadata.csv', index=False)
print("정리된 메타데이터 저장: cleaned_sliding_windows_metadata.csv")

# 새로운 시퀀스 데이터 저장
np.save('processed_sliding_windows_sequences.npy', new_sequences_array)
print("처리된 시퀀스 데이터 저장: processed_sliding_windows_sequences.npy")

# 피처 이름 저장
with open('feature_names.txt', 'w') as f:
    for feature in new_feature_cols:
        f.write(feature + '\n')
print("피처 이름 저장: feature_names.txt")

# ===== 8. 최종 요약 =====
print(f"\n===== 최종 처리 요약 =====")
print(f"원본 데이터: {original_data.shape[0]:,}행 × {original_data.shape[1]}열")
print(f"처리된 데이터: {processed_data.shape[0]:,}행 × {processed_data.shape[1]}열")
print(f"제거된 컬럼 수: {len(existing_cols_to_remove)}")
print(f"남은 피처 수: {len(new_feature_cols)}")
print(f"결측값 감소: {missing_before:,} → {missing_after:,}")
print(f"윈도우 수: {len(metadata_cleaned)}")
print(f"최종 시퀀스 형태: {new_sequences_array.shape[0]}윈도우 × {new_sequences_array.shape[1]}시간 × {new_sequences_array.shape[2]}피처")

# 각 파일별 결측값 현황
print(f"\n===== 최종 결측값 현황 =====")
final_missing = new_sequences_array.size - np.count_nonzero(~np.isnan(new_sequences_array))
final_total = new_sequences_array.size
print(f"최종 시퀀스 결측값: {final_missing:,} / {final_total:,} ({final_missing/final_total*100:.2f}%)")

print("\n모든 처리가 완료되었습니다!")

===== 생성된 파일 분석 =====
메타데이터 파일 크기: (352165, 10)
컬럼: ['subject_id', 'stay_id', 'window_id', 'obs_start_hour', 'obs_end_hour', 'pred_start_hour', 'pred_end_hour', 'obs_completeness', 'death_in_pred_window_old', 'death_in_pred_window_new']

메타데이터 샘플:
   subject_id   stay_id  window_id  obs_start_hour  obs_end_hour  \
0    10000690  37081114          0               0            18   
1    10000690  37081114          1               8            26   
2    10000690  37081114          2              16            34   
3    10000690  37081114          3              24            42   
4    10000690  37081114          4              32            50   

   pred_start_hour  pred_end_hour  obs_completeness  death_in_pred_window_old  \
0               18             24          0.752525                         0   
1               26             32          0.750842                         0   
2               34             40          0.791246                         0   
3               42 

### 결측값 보간 없는 수정 버전

In [1]:
#결측값 보간 없는 버전
import pandas as pd
import numpy as np

print("===== 생성된 파일 분석 =====")

# ===== 1. 생성된 파일들 로드 및 분석 =====
# 메타데이터 파일 로드
metadata = pd.read_csv('sliding_windows_metadata.csv')
print(f"메타데이터 파일 크기: {metadata.shape}")
print(f"컬럼: {list(metadata.columns)}")
print("\n메타데이터 샘플:")
print(metadata.head())

# 시퀀스 데이터 로드
sequences = np.load('sliding_windows_sequences.npy')
print(f"\n시퀀스 데이터 형태: {sequences.shape}")
print(f"시퀀스 데이터 타입: {sequences.dtype}")
print(f"각 윈도우 크기: {sequences.shape[1]}시간 × {sequences.shape[2]}피처")

# 메타데이터 통계
print(f"\n===== 메타데이터 통계 =====")
print(f"총 윈도우 수: {len(metadata)}")
print(f"환자 수: {metadata['subject_id'].nunique()}")
print(f"평균 관찰 완전성: {metadata['obs_completeness'].mean():.3f}")
print(f"기존 라벨 1 비율: {metadata['death_in_pred_window_old'].mean():.3f}")
print(f"새 라벨 1 비율: {metadata['death_in_pred_window_new'].mean():.3f}")

print("\n관찰 완전성 분포:")
print(metadata['obs_completeness'].describe())

# ===== 2. 원본 데이터 로드 =====
print("\n===== 원본 데이터 로드 =====")
original_data = pd.read_csv("hourly_bins/all_features_hourly_bins_inner.csv")
print(f"원본 데이터 크기: {original_data.shape}")

# 메타데이터 컬럼 제외하고 피처 컬럼만 추출
exclude_cols = ['subject_id', 'stay_id', 'hour_from_intime', 'hadm_id', 'bin_start', 'bin_end']
feature_cols = [c for c in original_data.columns if c not in exclude_cols]
print(f"처리할 피처 수: {len(feature_cols)}")

# 결측값 현황 확인
missing_before = original_data[feature_cols].isnull().sum().sum()
total_values = len(original_data) * len(feature_cols)
print(f"결측값 현황: {missing_before:,} / {total_values:,} ({missing_before/total_values*100:.2f}%)")

# ===== 3. 불필요한 컬럼 제거 =====
print("\n===== 컬럼 정리 =====")

# 처리된 데이터는 원본 데이터를 그대로 사용 (결측값 보간 제거)
processed_data = original_data.copy()

# 제거할 컬럼들 정의
cols_to_remove = ['temp_min', 'temp_max', 'gcs_min', 'gcs_max']

# old_label 관련 컬럼들도 찾아서 제거 (컬럼명에 'old' 또는 'old_label'이 포함된 것들)
old_label_cols = [col for col in processed_data.columns if 'old' in col.lower()]
cols_to_remove.extend(old_label_cols)

# 실제 존재하는 컬럼만 필터링
existing_cols_to_remove = [col for col in cols_to_remove if col in processed_data.columns]
print(f"제거할 컬럼들: {existing_cols_to_remove}")

# 컬럼 제거
if existing_cols_to_remove:
    processed_data = processed_data.drop(columns=existing_cols_to_remove)
    print(f"제거된 컬럼 수: {len(existing_cols_to_remove)}")
else:
    print("제거할 컬럼이 존재하지 않습니다.")

print(f"최종 데이터 크기: {processed_data.shape}")

# ===== 4. 메타데이터에서도 old_label 관련 컬럼 제거 =====
print("\n메타데이터에서 old_label 관련 컬럼 제거...")
metadata_old_cols = [col for col in metadata.columns if 'old' in col.lower()]
if metadata_old_cols:
    metadata_cleaned = metadata.drop(columns=metadata_old_cols)
    print(f"메타데이터에서 제거된 컬럼: {metadata_old_cols}")
else:
    metadata_cleaned = metadata.copy()
    print("메타데이터에서 제거할 old_label 관련 컬럼이 없습니다.")

# ===== 5. 처리된 데이터로 새로운 시퀀스 생성 =====
print("\n===== 새로운 시퀀스 데이터 생성 =====")

# 새로운 피처 컬럼 목록 (제거된 컬럼 반영)
new_exclude_cols = ['subject_id', 'stay_id', 'hour_from_intime', 'hadm_id', 'bin_start', 'bin_end']
new_feature_cols = [c for c in processed_data.columns if c not in new_exclude_cols]
print(f"새로운 피처 수: {len(new_feature_cols)}")

# 코호트 데이터 다시 로드
cohort = pd.read_csv("cohort.csv")
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])
cohort['deathtime'] = pd.to_datetime(cohort['deathtime'])

# 병합
processed_data_with_cohort = processed_data.merge(
    cohort[['subject_id','stay_id','intime','outtime','deathtime']],
    on=['subject_id','stay_id'], how='inner'
)

# 새로운 시퀀스 생성
obs_window_hours = 18
new_sequences = []

processed_grouped = processed_data_with_cohort.groupby('subject_id')

for _, row in metadata_cleaned.iterrows():
    subject_id = row['subject_id']
    obs_start = row['obs_start_hour']
    obs_end = row['obs_end_hour']
    
    # 해당 환자 데이터 가져오기
    subject_data = processed_grouped.get_group(subject_id).sort_values('hour_from_intime').reset_index(drop=True)
    
    # 관찰 구간 데이터 추출
    obs_mask = (subject_data['hour_from_intime'] >= obs_start) & \
               (subject_data['hour_from_intime'] < obs_end)
    obs_window = subject_data[obs_mask]
    
    # 시간별 데이터 생성
    hour_range = np.arange(obs_start, obs_end)
    obs_window_indexed = obs_window.set_index('hour_from_intime')
    
    sequence_data = []
    for hour in hour_range:
        if hour in obs_window_indexed.index:
            hour_features = obs_window_indexed.loc[hour, new_feature_cols].values
        else:
            # 해당 시간의 데이터가 없는 경우 NaN으로 채움
            hour_features = np.full(len(new_feature_cols), np.nan)
        sequence_data.append(hour_features.tolist())
    
    new_sequences.append(np.array(sequence_data))

new_sequences_array = np.array(new_sequences)
print(f"새로운 시퀀스 데이터 형태: {new_sequences_array.shape}")

# ===== 6. 최종 파일 저장 =====
print("\n===== 최종 파일 저장 =====")

# 처리된 원본 데이터 저장
processed_data.to_csv('processed_hourly_data.csv', index=False)
print("처리된 시간별 데이터 저장: processed_hourly_data.csv")

# 정리된 메타데이터 저장
metadata_cleaned.to_csv('cleaned_sliding_windows_metadata.csv', index=False)
print("정리된 메타데이터 저장: cleaned_sliding_windows_metadata.csv")

# 새로운 시퀀스 데이터 저장
np.save('processed_sliding_windows_sequences.npy', new_sequences_array)
print("처리된 시퀀스 데이터 저장: processed_sliding_windows_sequences.npy")

# 피처 이름 저장
with open('feature_names.txt', 'w') as f:
    for feature in new_feature_cols:
        f.write(feature + '\n')
print("피처 이름 저장: feature_names.txt")

# ===== 7. 최종 요약 =====
print(f"\n===== 최종 처리 요약 =====")
print(f"원본 데이터: {original_data.shape[0]:,}행 × {original_data.shape[1]}열")
print(f"처리된 데이터: {processed_data.shape[0]:,}행 × {processed_data.shape[1]}열")
print(f"제거된 컬럼 수: {len(existing_cols_to_remove)}")
print(f"남은 피처 수: {len(new_feature_cols)}")
print(f"결측값 현황: {missing_before:,} (보간 없이 유지)")
print(f"윈도우 수: {len(metadata_cleaned)}")
print(f"최종 시퀀스 형태: {new_sequences_array.shape[0]}윈도우 × {new_sequences_array.shape[1]}시간 × {new_sequences_array.shape[2]}피처")

# 각 파일별 결측값 현황
print(f"\n===== 최종 결측값 현황 =====")
final_missing = new_sequences_array.size - np.count_nonzero(~np.isnan(new_sequences_array))
final_total = new_sequences_array.size
print(f"최종 시퀀스 결측값: {final_missing:,} / {final_total:,} ({final_missing/final_total*100:.2f}%)")

print("\n모든 처리가 완료되었습니다!")

===== 생성된 파일 분석 =====
메타데이터 파일 크기: (352165, 10)
컬럼: ['subject_id', 'stay_id', 'window_id', 'obs_start_hour', 'obs_end_hour', 'pred_start_hour', 'pred_end_hour', 'obs_completeness', 'death_in_pred_window_old', 'death_in_pred_window_new']

메타데이터 샘플:
   subject_id   stay_id  window_id  obs_start_hour  obs_end_hour  \
0    10000690  37081114          0               0            18   
1    10000690  37081114          1               8            26   
2    10000690  37081114          2              16            34   
3    10000690  37081114          3              24            42   
4    10000690  37081114          4              32            50   

   pred_start_hour  pred_end_hour  obs_completeness  death_in_pred_window_old  \
0               18             24          0.752525                         0   
1               26             32          0.750842                         0   
2               34             40          0.791246                         0   
3               42 

## 시계열 + 정적 피쳐 통합

In [2]:
import pandas as pd
import numpy as np

# ===== 1. 파라미터 설정 =====
seq_file = 'processed_sliding_windows_sequences.npy'  # 기존 시계열 .npy
meta_file = 'cleaned_sliding_windows_metadata.csv'    # 메타데이터
cohort_file = 'cohort.csv'                            # 정적 피처 파일
static_cols = ['age', 'gender_M', 'gender_F']  # 병합할 정적 피처 컬럼명

# ===== 2. 시계열 데이터 로드 =====
print("시계열 데이터 로드...")
X_seq = np.load(seq_file)  # shape: (N, T, F_temporal)
N, T, F_temporal = X_seq.shape
print(f"시계열 shape: {X_seq.shape}")

# ===== 3. 메타데이터 & cohort 로드 =====
print("메타데이터, cohort 로드...")
meta_df = pd.read_csv(meta_file)   # (N행, subject_id & stay_id 포함)
cohort_df = pd.read_csv(cohort_file)

# ===== 4. cohort에서 필요한 정적 피처만 추출 =====
static_df = cohort_df[['subject_id', 'stay_id'] + static_cols]

# ===== 5. subject_id, stay_id 기준 병합 =====
print("정적 피처 병합...")
meta_with_static = meta_df.merge(
    static_df,
    on=['subject_id', 'stay_id'],
    how='left'
)

# ===== 6. 정적 피처 행렬 생성 =====
X_static = meta_with_static[static_cols].to_numpy()  # (N, F_static)
F_static = X_static.shape[1]

# ===== 7. 모든 시간스텝에 복사
print("정적 피처 시간축 복사...")
X_static_expanded = np.repeat(X_static[:, np.newaxis, :], T, axis=1)  # (N, T, F_static)

# ===== 8. 시계열 + 정적 concat
print("시계열 + 정적 피처 통합...")
X_combined = np.concatenate([X_seq, X_static_expanded], axis=2)  # (N, T, F_temporal+F_static)

# ===== 9. 라벨 추출 =====
y = meta_with_static['death_in_pred_window_new'].to_numpy()  # (N,)

# ===== 10. 저장 =====
np.save('tcn_input_combined.npy', X_combined)
np.save('tcn_labels.npy', y)
meta_with_static.to_csv('tcn_metadata_with_static.csv', index=False)

# ===== 11. 피처 이름 저장 =====
# 기존 temporal feature 이름 불러오기
with open('feature_names.txt', 'r') as f:
    temporal_feature_names = [line.strip() for line in f.readlines() if line.strip()]

# 정적 피처 포함
combined_feature_names = temporal_feature_names + static_cols

# 저장
with open('tcn_feature_names.txt', 'w') as f:
    f.write("\n".join(combined_feature_names))


print("\n===== 저장 완료 =====")
print(f"통합 입력 shape: {X_combined.shape}  # (윈도우 수, 시간스텝 수, 전체피처수)")
print(f"라벨 shape: {y.shape}")
print(f"피처 이름 수: {len(combined_feature_names)}")
print("파일:")
print(" - tcn_input_combined.npy          (X)")
print(" - tcn_labels.npy                  (y)")
print(" - tcn_metadata_with_static.csv    (메타데이터 + 정적)")
print(" - tcn_feature_names.txt           (피처 이름 목록)")


시계열 데이터 로드...
시계열 shape: (352165, 18, 29)
메타데이터, cohort 로드...
정적 피처 병합...
정적 피처 시간축 복사...
시계열 + 정적 피처 통합...

===== 저장 완료 =====
통합 입력 shape: (352165, 18, 32)  # (윈도우 수, 시간스텝 수, 전체피처수)
라벨 shape: (352165,)
피처 이름 수: 32
파일:
 - tcn_input_combined.npy          (X)
 - tcn_labels.npy                  (y)
 - tcn_metadata_with_static.csv    (메타데이터 + 정적)
 - tcn_feature_names.txt           (피처 이름 목록)


In [3]:
import numpy as np

# ===== 1. 파일 로드 =====
print("=== 파일 로드 ===")
X = np.load('tcn_input_combined.npy')
y = np.load('tcn_labels.npy')

# ===== 2. 기본 정보 출력 =====
print(f"X shape: {X.shape}  # (윈도우 수, 시간스텝 수, 전체 피처 수)")
print(f"y shape: {y.shape}  # (윈도우 수,)")

# ===== 3. NaN/inf 여부 확인 =====
print("\n=== 결측값 및 무한대 값 검증 ===")
nan_count_X = np.isnan(X).sum()
nan_count_y = np.isnan(y).sum()
inf_count_X = np.isinf(X).sum()
inf_count_y = np.isinf(y).sum()

print(f"X NaN 개수: {nan_count_X:,}")
print(f"X inf 개수: {inf_count_X:,}")
print(f"y NaN 개수: {nan_count_y:,}")
print(f"y inf 개수: {inf_count_y:,}")

# ===== 4. 값 범위 간단 확인 =====
print("\n=== 값 범위 ===")
print(f"X min: {np.nanmin(X):.4f}, X max: {np.nanmax(X):.4f}")
print(f"y unique 값: {np.unique(y)}")

# ===== 5. 샘플 데이터 출력 =====
print("\n=== 샘플 확인 (첫 윈도우 첫 타임스텝) ===")
print(X[0, 0, :])
print(f"y[0]: {y[0]}")



=== 파일 로드 ===
X shape: (352165, 18, 32)  # (윈도우 수, 시간스텝 수, 전체 피처 수)
y shape: (352165,)  # (윈도우 수,)

=== 결측값 및 무한대 값 검증 ===
X NaN 개수: 26,174,390
X inf 개수: 0
y NaN 개수: 0
y inf 개수: 0

=== 값 범위 ===
X min: -32.0000, X max: 8999090.0000
y unique 값: [0 1]

=== 샘플 확인 (첫 윈도우 첫 타임스텝) ===
[106.         105.         105.         107.          56.5
  50.          50.          63.          67.5         64.
  64.          71.          78.          80.          75.
  80.          36.5         36.5         98.          94.
  94.         100.          24.33333333  27.          23.
  27.          15.          15.           0.          86.
   0.           1.        ]
y[0]: 0


In [4]:
import numpy as np
import pandas as pd

print("===== NaN 결측값 확인 =====")

# 시퀀스 데이터 로드
sequences = np.load('tcn_input_combined.npy')
print(f"시퀀스 데이터 형태: {sequences.shape}")

# 1. NaN 값 존재 여부 확인
has_nan = np.isnan(sequences).any()
print(f"NaN 값 존재 여부: {has_nan}")

if has_nan:
    # 2. NaN 값의 개수와 비율
    total_values = sequences.size
    nan_count = np.isnan(sequences).sum()
    nan_ratio = nan_count / total_values * 100
    
    print(f"전체 값 개수: {total_values:,}")
    print(f"NaN 값 개수: {nan_count:,}")
    print(f"NaN 비율: {nan_ratio:.2f}%")
    
    # 3. 각 차원별 NaN 분포
    print(f"\n=== 차원별 NaN 분포 ===")
    print(f"윈도우별 NaN 개수 (상위 5개):")
    nan_per_window = np.isnan(sequences).sum(axis=(1,2))
    top_5_windows = np.argsort(nan_per_window)[-5:][::-1]
    for i, window_idx in enumerate(top_5_windows):
        print(f"  윈도우 {window_idx}: {nan_per_window[window_idx]:,}개")
    
    print(f"\n시간별 NaN 개수 (상위 5개):")
    nan_per_timestep = np.isnan(sequences).sum(axis=(0,2))
    top_5_timesteps = np.argsort(nan_per_timestep)[-5:][::-1]
    for i, time_idx in enumerate(top_5_timesteps):
        print(f"  시간 {time_idx}: {nan_per_timestep[time_idx]:,}개")
    
    print(f"\n피처별 NaN 개수 (상위 5개):")
    nan_per_feature = np.isnan(sequences).sum(axis=(0,1))
    top_5_features = np.argsort(nan_per_feature)[-5:][::-1]
    for i, feature_idx in enumerate(top_5_features):
        print(f"  피처 {feature_idx}: {nan_per_feature[feature_idx]:,}개")
    
    print(f"\n*** 결론: 마스킹이 필수입니다! ***")
    print("이유: NaN 값이 존재하면 PyTorch/TensorFlow에서 gradient 계산 시 문제가 발생합니다.")
    
else:
    print("\n*** 결론: 마스킹이 필수가 아닙니다 ***")
    print("이유: NaN 값이 없으므로 직접 모델에 입력 가능합니다.")

# 4. 샘플 데이터 확인 (처음 3개 윈도우의 첫 번째 시간 스텝)
print(f"\n=== 샘플 데이터 확인 ===")
print("처음 3개 윈도우의 첫 번째 시간 스텝:")
for i in range(min(3, sequences.shape[0])):
    sample_data = sequences[i, 0, :5]  # 첫 5개 피처만 출력
    print(f"윈도우 {i}: {sample_data}")
    if np.isnan(sample_data).any():
        print(f"  -> NaN 포함!")
    else:
        print(f"  -> 정상 데이터")

print("\n" + "="*50)

===== NaN 결측값 확인 =====
시퀀스 데이터 형태: (352165, 18, 32)
NaN 값 존재 여부: True
전체 값 개수: 202,847,040
NaN 값 개수: 26,174,390
NaN 비율: 12.90%

=== 차원별 NaN 분포 ===
윈도우별 NaN 개수 (상위 5개):
  윈도우 50158: 522개
  윈도우 130954: 498개
  윈도우 199100: 492개
  윈도우 146266: 478개
  윈도우 332756: 461개

시간별 NaN 개수 (상위 5개):
  시간 0: 1,583,524개
  시간 1: 1,508,025개
  시간 16: 1,498,222개
  시간 17: 1,494,653개
  시간 15: 1,476,256개

피처별 NaN 개수 (상위 5개):
  피처 28: 4,497,147개
  피처 27: 4,497,147개
  피처 26: 4,497,147개
  피처 16: 4,430,759개
  피처 17: 4,430,759개

*** 결론: 마스킹이 필수입니다! ***
이유: NaN 값이 존재하면 PyTorch/TensorFlow에서 gradient 계산 시 문제가 발생합니다.

=== 샘플 데이터 확인 ===
처음 3개 윈도우의 첫 번째 시간 스텝:
윈도우 0: [106.  105.  105.  107.   56.5]
  -> 정상 데이터
윈도우 1: [138. 138. 138. 138.  71.]
  -> 정상 데이터
윈도우 2: [114.  nan 114. 114.  56.]
  -> NaN 포함!



In [5]:
# 클래스 불균형 확인
import numpy as np
import pandas as pd

print("===== 환자별 클래스 불균형 확인 =====")

# 1. 메타데이터 로드 (환자 ID 정보 포함)
meta_df = pd.read_csv('tcn_metadata_with_static.csv')
print(f"전체 윈도우 수: {len(meta_df):,}")

# 2. 환자별 라벨 집계 (환자당 하나의 라벨만)
# 같은 환자의 여러 윈도우 중 하나라도 사망이면 사망으로 분류
patient_labels = meta_df.groupby('subject_id')['death_in_pred_window_new'].max()
print(f"고유 환자 수: {len(patient_labels):,}")

# 3. 환자별 클래스 개수 계산
survival_patients = np.sum(patient_labels == 0)  # 생존 환자
death_patients = np.sum(patient_labels == 1)     # 사망 환자

print(f"\n=== 환자별 클래스 개수 ===")
print(f"생존 환자: {survival_patients:,}명 ({survival_patients/len(patient_labels)*100:.1f}%)")
print(f"사망 환자: {death_patients:,}명 ({death_patients/len(patient_labels)*100:.1f}%)")

# 4. 불균형 비율 계산
if death_patients > 0:
    imbalance_ratio = survival_patients / death_patients
    print(f"\n불균형 비율: {imbalance_ratio:.1f}:1 (생존:사망)")
else:
    print("\n사망 환자가 없습니다!")

# 5. 윈도우 vs 환자 비교
y = np.load('tcn_labels.npy')
window_death_ratio = np.mean(y) * 100
patient_death_ratio = (death_patients / len(patient_labels)) * 100

print(f"\n=== 윈도우 vs 환자 비교 ===")
print(f"윈도우 기준 사망 비율: {window_death_ratio:.1f}%")
print(f"환자 기준 사망 비율: {patient_death_ratio:.1f}%")

# 6. 환자당 평균 윈도우 수
windows_per_patient = meta_df.groupby('subject_id').size()
avg_windows = windows_per_patient.mean()
print(f"환자당 평균 윈도우 수: {avg_windows:.1f}개")

# 7. 생존/사망 환자별 윈도우 수 비교
survival_patient_ids = patient_labels[patient_labels == 0].index
death_patient_ids = patient_labels[patient_labels == 1].index

survival_windows = meta_df[meta_df['subject_id'].isin(survival_patient_ids)].shape[0]
death_windows = meta_df[meta_df['subject_id'].isin(death_patient_ids)].shape[0]

print(f"\n=== 환자군별 윈도우 분포 ===")
print(f"생존 환자 윈도우: {survival_windows:,}개 (평균 {survival_windows/survival_patients:.1f}개/환자)")
print(f"사망 환자 윈도우: {death_windows:,}개 (평균 {death_windows/death_patients:.1f}개/환자)")

# 8. 불균형 정도 판단 (환자 기준)
minority_ratio = min(survival_patients, death_patients) / len(patient_labels)
print(f"\n=== 환자 기준 불균형 평가 ===")
if minority_ratio < 0.1:
    balance_level = "심각한 불균형"
elif minority_ratio < 0.2:
    balance_level = "중간 불균형"
elif minority_ratio < 0.4:
    balance_level = "약간 불균형"
else:
    balance_level = "균형적"

print(f"소수 클래스(환자) 비율: {minority_ratio*100:.1f}%")
print(f"평가: {balance_level}")

# 9. 추천 방법 (환자 기준)
print(f"\n=== 추천 처리 방법 ===")
if minority_ratio < 0.1:
    print("- 환자 레벨에서 SMOTE 적용")
    print("- 강한 클래스 가중치 필요")
    print("- Focal Loss 권장")
elif minority_ratio < 0.3:
    print("- 클래스 가중치 적용")
    print("- 환자별 균등 샘플링 고려")
else:
    print("- 현재 상태로 훈련 가능")
    print("- 가벼운 클래스 가중치 적용")

print("\n" + "="*50)

===== 환자별 클래스 불균형 확인 =====
전체 윈도우 수: 352,165
고유 환자 수: 42,979

=== 환자별 클래스 개수 ===
생존 환자: 41,703명 (97.0%)
사망 환자: 1,276명 (3.0%)

불균형 비율: 32.7:1 (생존:사망)

=== 윈도우 vs 환자 비교 ===
윈도우 기준 사망 비율: 0.4%
환자 기준 사망 비율: 3.0%
환자당 평균 윈도우 수: 8.2개

=== 환자군별 윈도우 분포 ===
생존 환자 윈도우: 333,495개 (평균 8.0개/환자)
사망 환자 윈도우: 18,670개 (평균 14.6개/환자)

=== 환자 기준 불균형 평가 ===
소수 클래스(환자) 비율: 3.0%
평가: 심각한 불균형

=== 추천 처리 방법 ===
- 환자 레벨에서 SMOTE 적용
- 강한 클래스 가중치 필요
- Focal Loss 권장

